## Step 1: Import the required libraries

In [4]:
# Import the core libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

## Step 2: Load your project data

In [5]:
# Data set path
path = r"/Users/martin/anaconda_projects/11-02-2025 Instacart Basket Analysis"

In [65]:
# import data (from 6.2)

df_listings = pd.read_csv(os.path.join(path, '02 data' , 'Original Data' , 'listingsTEST.csv'))

## Step 3: Load the GeoJSON file for Amsterdam neighbourhoods

In [67]:
# Correct file path
geo_path = os.path.join(path, '02 Data', 'Original Data', 'neighbourhoods.geojson')

# Load the GeoJSON file
with open(geo_path, encoding='utf-8') as f:
    amsterdam_geo = json.load(f)


## Step 4: Prepare the Data for Mapping

Now we create two aggregations based on my hypotheses:

1. Average price per neighborhood

2. Availability or listing count per neighborhood

In [20]:
# Group by neighbourhood and calculate average price
price_by_neighbourhood = df_listings.groupby('neighbourhood')['price'].mean().reset_index()

# OPTIONAL: Round the prices for nicer display
price_by_neighbourhood['price'] = price_by_neighbourhood['price'].round(2)

# Display the result
price_by_neighbourhood.head()


neighbourhood   price
0         Bijlmer-Centrum  137.68
1            Bijlmer-Oost  137.71
2           Bos en Lommer  208.83
3  Buitenveldert - Zuidas  778.41
4            Centrum-Oost  312.05

In [43]:
# Group by neighbourhood and count number of listings
listings_by_neighbourhood = df['neighbourhood'].value_counts().reset_index()
listings_by_neighbourhood.columns = ['neighbourhood', 'listing_count']

# Display the result
listings_by_neighbourhood.head()


neighbourhood  listing_count
0  Amsterdam, Noord-Holland, Netherlands           4470
1  Amsterdam, North Holland, Netherlands            530
2             Amsterdam, NH, Netherlands            197
3                 Amsterdam, Netherlands             51
4   Amsterdam, Nord Holland, Netherlands             13

## Step 5: Wrangle the Data (prepare relevant columns)

In [22]:
# Inspect the structure of the DataFrame
df_listings.info()
df_listings.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10075 entries, 0 to 10074
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              10075 non-null  int64  
 1   name                            10075 non-null  object 
 2   host_id                         10075 non-null  int64  
 3   host_name                       10058 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   10075 non-null  object 
 6   latitude                        10075 non-null  float64
 7   longitude                       10075 non-null  float64
 8   room_type                       10075 non-null  object 
 9   price                           5911 non-null   float64
 10  minimum_nights                  10075 non-null  int64  
 11  number_of_reviews               10075 non-null  int64  
 12  last_review                     

id                                               name  host_id  \
0   2818          Quiet Garden View Room & Super Fast Wi-Fi     3159   
1  27886  Romantic, stylish B&B houseboat in canal district    97647   
2  28871                            Comfortable double room   124245   
3  29051                            Comfortable single room   124245   
4  44391    Quiet 2-bedroom Amsterdam city centre apartment   194779   

  host_name  neighbourhood_group                           neighbourhood  \
0    Daniel                  NaN  Oostelijk Havengebied - Indische Buurt   
1      Flip                  NaN                            Centrum-West   
2     Edwin                  NaN                            Centrum-West   
3     Edwin                  NaN                            Centrum-Oost   
4       Jan                  NaN                            Centrum-Oost   

   latitude  longitude        room_type  price  minimum_nights  \
0  52.36435    4.94358     Private room   66.0               3   
1  52.38761    4.89188     Private room    NaN               3   
2  52.36775    4.89092     Private room   75.0               2   
3  52.36584    4.89111     Private room   59.0               2   
4  52.37168    4.91471  Entire home/apt    NaN               3   

   number_of_reviews last_review  reviews_per_month  \
0                398  2025-01-03               2.05   
1                294  2025-02-17               1.84   
2                677  2025-02-28               3.83   
3                794  2025-03-02               4.67   
4                 42  2022-08-20               0.24   

   calculated_host_listings_count  availability_365  number_of_reviews_ltm  \
0                               1                10                     31   
1                               1                88                     27   
2                               2                84                     94   
3                               2                84                     85   
4                               1                 0                      0   

                    license  
0  0363 5F3A 5684 6750 D14D  
1  0363 974D 4986 7411 88D8  
2  0363 607B EA74 0BD8 2F6F  
3  0363 607B EA74 0BD8 2F6F  
4  0363 E76E F06A C1DD 172C

### Cut out relevant columns

- You can now create a new DataFrame that only contains the columns you want to analyze:

In [26]:
# Select only the relevant columns for analysis
df_listings = df_listings[['neighbourhood', 'price', 'availability_365']]


In [28]:
# Check the output 
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10075 entries, 0 to 10074
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   neighbourhood     10075 non-null  object 
 1   price             5911 non-null   float64
 2   availability_365  10075 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 236.3+ KB


## Step 6: Clean the Data (handle missing and extreme values)

### 6.1: Let’s clean it step-by-step:

In [30]:
# Check for missing values
df_listings.isnull().sum()


neighbourhood          0
price               4164
availability_365       0
dtype: int64

### Fill missing values in price with the median

In [33]:
# Fill missing price values with the median
df_listings['price'] = df_listings['price'].fillna(df_listings['price'].median()) # Since price is important, I don't want to lose too much data. So:


In [35]:
# Check again for missing values
df_listings.isnull().sum()


neighbourhood       0
price               0
availability_365    0
dtype: int64

### 6.2: Check for Extreme Values

- Now we check whether there are extremely high outliers in the price column that could later distort the choropleth.

In [37]:
# Show the top 10 most expensive listings
df_listings['price'].sort_values(ascending=False).head(10)


2171    14404.0
4166    10000.0
429     10000.0
9815     9999.0
7138     9999.0
7136     9999.0
5230     9000.0
7137     7999.0
6650     6356.0
3933     5135.0
Name: price, dtype: float64

### 6.3: Remove extreme values from the price column

In [39]:
# Remove extreme values (e.g. listings with price > 5000)
df_listings = df_listings[df_listings['price'] < 5000]

# Check again to confirm
df_listings['price'].sort_values(ascending=False).head(10)


7282    3038.0
4960    2600.0
5115    2600.0
5523    2291.0
5641    1995.0
6468    1800.0
7384    1760.0
1699    1754.0
442     1650.0
7198    1541.0
Name: price, dtype: float64

## Step 7: Create a Choropleth Map

### Data Preparation – Calculate Mean Price per Neighbourhood

In [41]:
# Group by neighbourhood and calculate the mean price
price_by_neighbourhood = df_listings.groupby('neighbourhood')['price'].mean().reset_index()

# Round values for nicer display
price_by_neighbourhood['price'] = price_by_neighbourhood['price'].round(2)

# Display the result
price_by_neighbourhood.head()


neighbourhood   price
0         Bijlmer-Centrum  167.77
1            Bijlmer-Oost  167.42
2           Bos en Lommer  208.92
3  Buitenveldert - Zuidas  239.93
4            Centrum-Oost  248.58

### Data Cleaning – Align Neighbourhood Names for Mapping

In [134]:
# Step 1: Extract all neighbourhood names from GeoJSON
geo_names = [feature['properties']['neighbourhood'] for feature in amsterdam_geo['features']]

# Step 2: Filter DataFrame to only include rows where neighbourhood is in GeoJSON
price_by_neighbourhood = price_by_neighbourhood[
    price_by_neighbourhood['neighbourhood'].isin(geo_names)
]

# Optional: Check result
price_by_neighbourhood.shape


(0, 2)

In [47]:
print(price_by_neighbourhood.columns)


Index(['neighbourhood', 'price'], dtype='object')


In [49]:
price_by_neighbourhood = price_by_neighbourhood.reset_index(drop=True)


In [51]:
print(price_by_neighbourhood.isnull().sum())


neighbourhood    0
price            0
dtype: int64


In [53]:
print(price_by_neighbourhood.dtypes)

neighbourhood     object
price            float64
dtype: object


In [55]:
price_by_neighbourhood.head()


neighbourhood   price
0         Bijlmer-Centrum  167.77
1            Bijlmer-Oost  167.42
2           Bos en Lommer  208.92
3  Buitenveldert - Zuidas  239.93
4            Centrum-Oost  248.58

### Create Choropleth Map – Average Airbnb Price per Neighbourhood

In [89]:
# Import folium for geospatial plotting
import folium
import json

# Load GeoJSON file with neighbourhood shapes
with open("/Users/martin/anaconda_projects/11-02-2025 Instacart Basket Analysis/02 Data/Original Data/neighbourhoods.geojson", encoding='utf-8') as f:
    amsterdam_geo = json.load(f)

# Create a folium map centered on Amsterdam
map_price = folium.Map(location=[52.3676, 4.9041], zoom_start=11)

# Add the choropleth layer
folium.Choropleth(
    geo_data=amsterdam_geo,
    data=price_by_neighbourhood,
    columns=['neighbourhood', 'price'],
    key_on='feature.properties.neighbourhood',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='white',
    legend_name='Average Airbnb Price (€)'
).add_to(map_price)

# Display the map
map_price

In [97]:
# save map output

map_price.save("choropleth_map.html")

## Step 8: Interpretation of results

## Choropleth Map: Average Airbnb Price per Neighbourhood

The choropleth map visualizes the **average Airbnb price** per neighbourhood in Amsterdam. We can observe a clear trend:

- **Central neighbourhoods**, especially in the city center (e.g. Centrum-West, Centrum-Oost), show the **highest average prices**.
- **Neighbourhoods further from the city center**, such as in the north or outer districts, have **lower average prices**.
- There are a few exceptions where non-central districts also show relatively high prices, possibly due to high-end listings or waterfront locations.

### Answer to Research Question

This analysis directly supports the first research question:

> *Does the average price vary by neighbourhood in Amsterdam?*

**Yes – the price clearly varies**, and is highest in central, tourist-heavy districts.

This also confirms the initial hypothesis:

> *“The average price varies depending on the district in Amsterdam - higher prices are assumed in central, touristy districts.”*

### New Research Questions

The results lead to further questions such as:

- What factors (besides location) influence the high prices in some outer neighbourhoods?
- How does availability or number of listings relate to price across districts?
- Are Superhosts more common in high-price areas?

These could be explored in the next steps of the project.



In [95]:
# Export data to csv

df_listings.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'listings_prepared_geo.csv'), index=False)